In [8]:
from gurobipy import *

#create new model    
m = Model('Five Period Inventory Problem')
            
#create production variables
x1 = m.addVar(vtype=GRB.CONTINUOUS, name="x1", lb=0, ub=GRB.INFINITY, obj=22)
x2 = m.addVar(vtype=GRB.CONTINUOUS, name="x2", lb=0, ub=GRB.INFINITY, obj=42)
x3 = m.addVar(vtype=GRB.CONTINUOUS, name="x3", lb=0, ub=GRB.INFINITY, obj=100)
x4 = m.addVar(vtype=GRB.CONTINUOUS, name="x4", lb=0, ub=GRB.INFINITY, obj=1000)
x5 = m.addVar(vtype=GRB.CONTINUOUS, name="x5", lb=0, ub=GRB.INFINITY, obj=450)

#create inventory variables
i01 = m.addVar(vtype=GRB.CONTINUOUS, name="i01", lb=0, ub=GRB.INFINITY, obj=0)
i12 = m.addVar(vtype=GRB.CONTINUOUS, name="i12", lb=0, ub=GRB.INFINITY, obj=5)
i23 = m.addVar(vtype=GRB.CONTINUOUS, name="i23", lb=0, ub=GRB.INFINITY, obj=11)
i34 = m.addVar(vtype=GRB.CONTINUOUS, name="i34", lb=0, ub=GRB.INFINITY, obj=4)
i45 = m.addVar(vtype=GRB.CONTINUOUS, name="i45", lb=0, ub=GRB.INFINITY, obj=0)
i56 = m.addVar(vtype=GRB.CONTINUOUS, name="i56", lb=0, ub=GRB.INFINITY, obj=0)

#integrate new variables    
m.update()
                    
#add constraints

#production capacity constraints
m.addConstr(x1 <= 40, "c1")
m.addConstr(x2 <= 100, "c2")
m.addConstr(x3 <= 90, "c3")
m.addConstr(x4 <= 500, "c4")
m.addConstr(x5 <= 1000, "c5")

#inventory capacity constraints
m.addConstr(i01 == 0, "c6")
m.addConstr(i12 <= 50, "c7")
m.addConstr(i23 <= 50, "c8")
m.addConstr(i34 <= 100, "c9")
m.addConstr(i45 <= 20, "c10")
m.addConstr(i56 <= 1000, "c11")

#balance constraints
m.addConstr(i01 + x1 - i12 == 10, "demand1")
m.addConstr(i12 + x2 - i23 == 80, "demand2")
m.addConstr(i23 + x3 - i34 == 100, "demand3")
m.addConstr(i34 + x4 - i45 == 20, "demand4")
m.addConstr(i45 + x5 - i56 == 440, "demand5")
              
#optimize    
m.optimize()
print ('Model status:', m.status)

#write LP
m.write('Linear Program.lp')

#print decision variables
for v in m.getVars():
    print (v.varName, v.x)
                
#print objective value
print ('Obj: ', m.objVal)



Optimize a model with 16 rows, 11 columns and 26 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+00, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+03]
Presolve removed 13 rows and 5 columns
Presolve time: 0.01s
Presolved: 3 rows, 6 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9723264e+05   9.010200e+01   0.000000e+00      0s
       3    2.0394000e+05   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds
Optimal objective  2.039400000e+05
Model status: 2
x1 40.0
x2 100.0
x3 90.0
x4 0.0
x5 420.0
i01 0.0
i12 30.0
i23 50.0
i34 40.0
i45 20.0
i56 0.0
Obj:  203940.0


In [9]:
from gurobipy import *

#create new model    
m = Model('Five Period Inventory Problem')


production_costs = [22, 42, 100, 1000, 450]
production_capacities = [40, 100, 90, 500, 1000]

inventory_costs = [0, 5, 11, 4, 0, 0]
inventory_capacities = [50, 50, 100, 20, 1000]

demand = [10, 80, 100, 20, 440]

#create production variables
production_variables = []
for i in range(len(production_costs)):
    production_variables.append(m.addVar(vtype=GRB.CONTINUOUS, name="x{}".format(i+1), obj = production_costs[i]))

#create inventory variables
inventory_variables = []
for i in range(len(inventory_costs)):
    inventory_variables.append(m.addVar(vtype=GRB.CONTINUOUS, name="i{}{}".format(i,i+1), obj = inventory_costs[i]))

#integrate new variables    
m.update()
                    
#production capacity constraints
for i in range(len(production_costs)):
    m.addConstr(production_variables[i] <= production_capacities[i])

#inventory capacity constraints
for i in range(len(inventory_capacities)):
    m.addConstr(inventory_variables[i+1] <= inventory_capacities[i])
    
m.addConstr(inventory_variables[0] == 0)

#demand constraints
for i in range(len(production_costs)):
    m.addConstr(inventory_variables[i] + production_variables[i] - inventory_variables[i+1]  == 
                demand[i])

#optimize    
m.optimize()
print ('Model status:', m.status)

#write LP
m.write('Linear Program1.lp')

#print decision variables
for v in m.getVars():
    print (v.varName, v.x)
                
#print objective value
print ('Obj: ', m.objVal)

Optimize a model with 16 rows, 11 columns and 26 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+00, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+03]
Presolve removed 13 rows and 5 columns
Presolve time: 0.01s
Presolved: 3 rows, 6 columns, 8 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9723264e+05   9.010200e+01   0.000000e+00      0s
       3    2.0394000e+05   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds
Optimal objective  2.039400000e+05
Model status: 2
x1 40.0
x2 100.0
x3 90.0
x4 0.0
x5 420.0
i01 0.0
i12 30.0
i23 50.0
i34 40.0
i45 20.0
i56 0.0
Obj:  203940.0
